In [1]:
import geopandas as gpd
import json
import ssl 
import geemap
ssl._create_default_https_context = ssl._create_unverified_context
import ee

# Trigger the authentication flow.
# Don't need to run the below as I am alreay authenticated
# ee.Authenticate()

# Initialize the library.
ee.Initialize()

In [2]:
from typing import Dict, Optional
def ee_geo_compatible(geoJSON_input: Dict):
    geoJSON_output = dict(geoJSON_input)
    try:
        if geoJSON_input["type"]=="Polygon":
            check_coords = geoJSON_input["coordinates"][0][0]
            if len(check_coords)>2:
                print("Forcefully making coordinates compatible ee.Geometry as current len>2:", check_coords)
                new_coords = [[x[0:2] for x in y] for y in geoJSON_input["coordinates"]]
                geoJSON_output["coordinates"] = new_coords
        else:
            print("currently unknown handling for type:", geoJSON_input["type"])
        return geoJSON_output
    except:
        print("Dict passed does not follow geoJSON format")
        return None

In [3]:
# Read in KML file
import geopandas as gpd
from fiona.drvsupport import supported_drivers
# Path to your KML file
kml_file_path = 'townsville.kml'

# Read the KML file using geopandas
supported_drivers['KML'] = 'rw'
my_map = gpd.read_file(kml_file_path, driver='KML')
kml_geo_json=json.loads(my_map.to_json())
new_kml_geo_json = dict(kml_geo_json)
# Initiate with no features
new_kml_geo_json["features"] = []
# display("original geojson")
# display(kml_geo_json)
for feat in kml_geo_json["features"]:
    new_feat = dict(feat)
    new_feat["geometry"] = ee_geo_compatible(feat["geometry"])
    if new_feat["geometry"]:
        try:
            ee.Geometry(new_feat["geometry"])
            print("succesfully passed to ee.Geometry")
            new_kml_geo_json["features"] += [new_feat]
        except:
            print("Unable to pass into ee.Geometry asses below feature")
            print(new_feat)
            
# display("updated geoJSON")
# display(new_kml_geo_json)
geojson_fc = ee.FeatureCollection(new_kml_geo_json)
# display(geojson_fc.geometry())

Forcefully making coordinates compatible ee.Geometry as current len>2: [146.6789497574668, -19.23247934166526, 0.0]
succesfully passed to ee.Geometry


In [28]:
import pandas as pd
# Initialise the Geemap

m = geemap.Map()
poly = geojson_fc.geometry()
poly_area = poly.area().getInfo()
m.centerObject(poly, 12)
display(m)

# Standard Functions 


def class_val_dict(val_list, class_name_list):
    output_dict = {}
    if len(val_list)==len(class_name_list):
        for i in range(len(val_list)):
            output_dict[f"{val_list[i]}"]=class_name_list[i]
    return output_dict

def area_sum_by_class_func(item):
    areaDict = ee.Dictionary(item)
    classNumber = ee.Number(areaDict.get('class')).format()
    area = ee.Number(areaDict.get('sum')).round()
    return ee.List([classNumber, area])

def display_area_by_class(dataset, poly_clip, class_name_dict = {}):
    data=pd.DataFrame()
    areaImage = ee.Image.pixelArea().addBands(dataset.clip(poly_clip))
    areas = areaImage.reduceRegion(
        reducer = ee.Reducer.sum().group(
            groupField= 1,
            groupName= 'class',
        ),
        geometry= poly,
        scale= 1,
        maxPixels= 1e10
        )
    classAreas = ee.List(areas.get('groups'))
    classAreaLists = classAreas.map(area_sum_by_class_func)
    result = ee.Dictionary(classAreaLists.flatten())
    tot_perc = 0
    for val, area in result.getInfo().items():
        perc =  area/poly_area*100
        tot_perc += perc
        if class_name_dict:
            # add code here
            new_row=pd.DataFrame({
                'class_name': [class_name_dict[f"{val}"]], 
                'class_val': [val], 
                'area': [round(area, 2)],
                'perc': [round(perc, 2)], 
                'cum_perc': [round(tot_perc,2)],
            })
            data = pd.concat([data,new_row])
            # print('class Name:', class_name_dict[f"{val}"], 'class Val:', val, 'Area:', round(area, 2), 'Perc:', round(perc, 2), 'Cum_Perc:', round(tot_perc,2))
        else:
            new_row=pd.DataFrame({
                'class_name': ["NA"], 
                'class_val': [val], 
                'area': [round(area, 2)],
                'perc': [round(perc, 2)], 
                'cum_perc': [round(tot_perc,2)],
            })
            data = pd.concat([data,new_row])
            # print('class Val:', val, 'Area:', round(area, 2), 'Perc:', round(perc, 2), 'Cum_Perc:', round(tot_perc,2))
    if result.getInfo():
        return data

# import land cover maps 2009, 2015, 2019, 2021

# 2009 GlobCover
dataset_2009 = ee.Image('ESA/GLOBCOVER_L4_200901_200912_V2_3').select(
    'landcover'
)
visualization = {
}
m.add_ee_layer(dataset_2009.clip(poly), visualization, '2009_Landcover', shown=True)
m.add_legend(title='2009_Landcover', builtin_legend='GLOBCOVER', layer_name='2009_Landcover')
m.show_layer(name='2009_Landcover' , show=False)
# 2009 GlobCover Area 
display('## 2009 GlobCover') 
display(display_area_by_class(dataset_2009, poly))

# 2015 Copernicus
visualization = {
    'bands': ['discrete_classification'],
}
dataset_2015 = ee.Image('COPERNICUS/Landcover/100m/Proba-V-C3/Global/2015').select(
    'discrete_classification'
)
m.add_ee_layer(dataset_2015.clip(poly), visualization, '2015_Landcover', shown=False)
m.add_legend(title='2015_Landcover', builtin_legend='COPERNICUS/Landcover/100m/Proba-V/Global', layer_name='2015_Landcover')
m.show_layer(name='2015_Landcover' , show=False)
# 2015 Copernicus Area 
display('## 2015 Copernicus') 
new_dict = class_val_dict(dataset_2015.get("discrete_classification_class_values").getInfo(), dataset_2015.get("discrete_classification_class_names").getInfo())
display(display_area_by_class(dataset_2015, poly, new_dict))


# 2019 Copernicus
dataset_2019 = ee.Image('COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019').select(
    'discrete_classification'
)
visualization = {
    'bands': ['discrete_classification'],
}
m.add_ee_layer(dataset_2019.clip(poly), visualization, '2019_Landcover', shown=False)
m.add_legend(title='2019_Landcover', builtin_legend='COPERNICUS/Landcover/100m/Proba-V/Global', layer_name='2019_Landcover')
m.show_layer(name='2019_Landcover' , show=False)
# 2019 Copernicus Area 
display('## 2019 Copernicus') 
new_dict = class_val_dict(dataset_2019.get("discrete_classification_class_values").getInfo(), dataset_2019.get("discrete_classification_class_names").getInfo())

display(display_area_by_class(dataset_2019, poly, new_dict))


# 2021 ESA World Cover
dataset_2021 = ee.ImageCollection('ESA/WorldCover/v200').first().select('Map')
visualization = {
    'bands': ['Map'],
}
m.add_ee_layer(dataset_2021.clip(poly), visualization, '2021_Landcover', shown=False)
m.add_legend(title='2021_Landcover', builtin_legend='ESA_WorldCover', layer_name='2021_Landcover')
# 2021 ESA World Cover Area 
display('## 2021 ESA World Cover') 
new_dict = class_val_dict(dataset_2021.get("Map_class_values").getInfo(), dataset_2021.get("Map_class_names").getInfo())
display(display_area_by_class(dataset_2021, poly, new_dict))


# Add historical satalite views 
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)

rgb_vis = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}
sat_dict = {}
for yr in ['2015','2017','2019', '2021', '2023']:
    sat_dict[f"dataset_sat_{yr}"] = (
        ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
        .filterDate(f'{yr}-01-01', f'{yr}-12-31')
        # Pre-filter to get less cloudy granules.
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
        .map(mask_s2_clouds)
    )

    m.add_layer(sat_dict[f"dataset_sat_{yr}"].median().clip(poly), rgb_vis, f'{yr}_Satalite', shown=False)


Map(center=[-19.281796805191107, 146.76263396612705], controls=(WidgetControl(options=['position', 'transparen…

'## 2009 GlobCover'

,class_name,class_val,area,perc,cum_perc
0,NA,110,15431292,8.59,8.59
0,NA,120,2447744,1.36,9.96
0,NA,130,33684185,18.76,28.72
0,NA,140,2066762,1.15,29.87
0,NA,150,5213411,2.90,32.77
0,NA,190,108454297,60.40,93.17
0,NA,20,89670,0.05,93.22
0,NA,200,269889,0.15,93.37
0,NA,210,8013124,4.46,97.84
0,NA,30,179634,0.10,97.94


'## 2015 Copernicus'

,class_name,class_val,area,perc,cum_perc
0,"Closed forest, evergreen broad leaf. Tree cano...",112,3389396,1.89,1.89
0,"Closed forest, deciduous broad leaf. Tree cano...",114,7204429,4.01,5.90
0,"Closed forest, not matching any of the other d...",116,310196,0.17,6.07
0,"Open forest, evergreen broad leaf. Top layer- ...",122,133244,0.07,6.15
0,"Open forest, deciduous broad leaf. Top layer- ...",124,25516154,14.21,20.36
0,"Open forest, not matching any of the other def...",126,8592278,4.79,25.14
0,Shrubs. Woody perennial plants with persistent...,20,761373,0.42,25.57
0,"Oceans, seas. Can be either fresh or salt-wate...",200,9001258,5.01,30.58
0,Herbaceous vegetation. Plants without persiste...,30,35213469,19.61,50.19
0,Cultivated and managed vegetation / agricultur...,40,450415,0.25,50.44


'## 2019 Copernicus'

,class_name,class_val,area,perc,cum_perc
0,"Closed forest, evergreen broad leaf. Tree cano...",112,3327135,1.85,1.85
0,"Closed forest, deciduous broad leaf. Tree cano...",114,7240307,4.03,5.89
0,"Closed forest, not matching any of the other d...",116,310196,0.17,6.06
0,"Open forest, evergreen broad leaf. Top layer- ...",122,97648,0.05,6.11
0,"Open forest, deciduous broad leaf. Top layer- ...",124,25287105,14.08,20.20
0,"Open forest, not matching any of the other def...",126,8461719,4.71,24.91
0,Shrubs. Woody perennial plants with persistent...,20,637463,0.36,25.26
0,"Oceans, seas. Can be either fresh or salt-wate...",200,9001258,5.01,30.28
0,Herbaceous vegetation. Plants without persiste...,30,34490109,19.21,49.48
0,Cultivated and managed vegetation / agricultur...,40,432762,0.24,49.73


'## 2021 ESA World Cover'

,class_name,class_val,area,perc,cum_perc
0,Tree cover,10,38325854,21.34,21.34
0,Shrubland,20,42751,0.02,21.37
0,Grassland,30,51868381,28.89,50.26
0,Cropland,40,408532,0.23,50.48
0,Built-up,50,69312641,38.60,89.09
0,Bare / sparse vegetation,60,4383107,2.44,91.53
0,Permanent water bodies,80,9815750,5.47,96.99
0,Herbaceous wetland,90,315213,0.18,97.17
0,Mangroves,95,4541764,2.53,99.70


In [16]:
if m.draw_features:
    new_drawn_fc = ee.FeatureCollection(m.draw_features)
    new_poly = new_drawn_fc.geometry()
    new_poly_area = new_poly.area().getInfo()
    # 2021 ESA World Cover
    dataset_new = ee.ImageCollection('ESA/WorldCover/v200').first()
    visualization = {
        'bands': ['Map'],
    }
    
    # 2009 GlobCover Area 
    display('## 2009 GlobCover Drawn Poly') 
    dataset_2009 = ee.Image('ESA/GLOBCOVER_L4_200901_200912_V2_3').select(
        'landcover'
    )
    areaImage = ee.Image.pixelArea().addBands(dataset_2009.clip(new_poly))
    areas = areaImage.reduceRegion(
        reducer = ee.Reducer.sum().group(
            groupField= 1,
            groupName= 'class',
        ),
        geometry= new_poly,
        scale= 1,
        maxPixels= 1e10
        )
    classAreas = ee.List(areas.get('groups'))
    classAreaLists = classAreas.map(area_sum_by_class_func)
    result = ee.Dictionary(classAreaLists.flatten())
    tot_perc = 0
    for val, area in result.getInfo().items():
        perc =  area/new_poly_area*100
        tot_perc += perc
        print('class:', val, 'Area:', area, 'Perc:', perc, 'Cum_Perc:', tot_perc)

    # 2021 ESA World Cover Area 
    display('## 2021 ESA World Cover') 
    dataset_2021 = ee.ImageCollection('ESA/WorldCover/v200').first().select('Map')
    areaImage = ee.Image.pixelArea().addBands(dataset_2021.clip(new_poly))
    areas = areaImage.reduceRegion(
        reducer = ee.Reducer.sum().group(
            groupField= 1,
            groupName= 'class',
        ),
        geometry= new_poly,
        scale= 1,
        maxPixels= 1e10
        )
    classAreas = ee.List(areas.get('groups'))
    classAreaLists = classAreas.map(area_sum_by_class_func)
    result = ee.Dictionary(classAreaLists.flatten())
    tot_perc = 0
    for val, area in result.getInfo().items():
        perc =  area/new_poly_area*100
        tot_perc += perc
        print('class:', val, 'Area:', area, 'Perc:', perc, 'Cum_Perc:', tot_perc)

'## 2009 GlobCover Drawn Poly'

class: 110 Area: 625482 Perc: 7.002694648731368 Cum_Perc: 7.002694648731368
class: 120 Area: 345880 Perc: 3.872360875457976 Cum_Perc: 10.875055524189344
class: 130 Area: 5203685 Perc: 58.25877819534965 Cum_Perc: 69.133833719539
class: 150 Area: 89940 Perc: 1.0069392192051878 Cum_Perc: 70.14077293874419
class: 190 Area: 1102200 Perc: 12.339875554902802 Cum_Perc: 82.480648493647
class: 210 Area: 22945 Perc: 0.2568848163738385 Cum_Perc: 82.73753331002084
class: 40 Area: 1514923 Perc: 16.960589090237722 Cum_Perc: 99.69812240025855


'## 2021 ESA World Cover'

class: 10 Area: 1242570 Perc: 13.911412781941184 Cum_Perc: 13.911412781941184
class: 20 Area: 423 Perc: 0.004735771511271898 Cum_Perc: 13.916148553452455
class: 30 Area: 4459439 Perc: 49.92644012400671 Cum_Perc: 63.84258867745916
class: 40 Area: 77227 Perc: 0.8646085732884038 Cum_Perc: 64.70719725074757
class: 50 Area: 460949 Perc: 5.160636270329243 Cum_Perc: 69.86783352107682
class: 60 Area: 499249 Perc: 5.589430712129984 Cum_Perc: 75.4572642332068
class: 80 Area: 80020 Perc: 0.8958781000755962 Cum_Perc: 76.35314233328239
class: 90 Area: 17906 Perc: 0.20046979829984538 Cum_Perc: 76.55361213158224
class: 95 Area: 2067272 Perc: 23.144510268676306 Cum_Perc: 99.69812240025854


In [34]:
# ee.Image('ESA/GLOBCOVER_L4_200901_200912_V2_3').select(
#     'landcover'
# ).getInfo()


ee.ImageCollection('ESA/WorldCover/v200').first().select('Map').get("Map_class_names").getInfo()


# discrete_classification_class_names

# discrete_classification_class_values

['Tree cover',
 'Shrubland',
 'Grassland',
 'Cropland',
 'Built-up',
 'Bare / sparse vegetation',
 'Snow and ice',
 'Permanent water bodies',
 'Herbaceous wetland',
 'Mangroves',
 'Moss and lichen']